In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import googlemaps
from datetime import datetime
import re
from geopy.geocoders import Nominatim

In [3]:
hospital = pd.read_csv('./data/서울특별시 병원 인허가 정보.csv')
hospital

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,진료과목내용명,지정취소일자,완화의료지정형태,완화의료담당부서명,구급차특수,구급차일반,총인원,구조사수,허가병상수,최초지정일자
0,3000000,PHMA220083000034021200001,20080624,NaN,3,폐업,3,폐업,20181001.0,NaN,...,한방부인과 침구과 사상체질과 한방재활의학과 한방신경정신과 한방안?이비인후?피부과 한...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
1,3000000,PHMA220103000034021200001,20080912,NaN,3,폐업,3,폐업,20111007.0,NaN,...,피부과 소아청소년과 이비인후과 외과 재활의학과 내과,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
2,3000000,PHMA219823000034021200001,19820924,NaN,3,폐업,3,폐업,20130108.0,NaN,...,영상의학과 내과,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
3,3000000,PHMA220063000034021200001,20060803,NaN,3,폐업,3,폐업,20120217.0,NaN,...,예방치과 구강병리과 구강악안면방사선과 구강내과 치과보존과 치주과 소아치과 치과교정과...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
4,3000000,PHMA219663000034021200001,19660119,NaN,3,폐업,3,폐업,20080930.0,NaN,...,치과 신경과 응급의학과 영상의학과 흉부외과 재활의학과 가정의학과 진단검사의학과 병리...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,3240000,PHMA220143240033021200001,20140204,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
844,3120000,PHMA220153120033021200003,20150811,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
845,3180000,PHMA220213180034021200001,20210928,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
846,3120000,PHMA218853120033021200001,19740325,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
hospital = hospital[hospital['상세영업상태명']== '영업중']
hospital

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,진료과목내용명,지정취소일자,완화의료지정형태,완화의료담당부서명,구급차특수,구급차일반,총인원,구조사수,허가병상수,최초지정일자
5,3000000,PHMA220043000034021200001,20040901,NaN,1,영업/정상,13,영업중,NaN,NaN,...,구강병리과 영상치의학과 구강내과 치과보존과 치주과 소아치과 치과교정과 치과보철과 구...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
7,3190000,PHMA220013190033021200001,20010307,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3220000,PHMA220203220033021200003,20200918,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3000000,PHMA219773000034021200001,19770401,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,3010000,PHMA220003010033021200001,20000810,NaN,1,영업/정상,13,영업중,NaN,NaN,...,치과보철과 치과교정과 소아치과 치주과 치과보존과,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,3240000,PHMA220143240033021200001,20140204,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
844,3120000,PHMA220153120033021200003,20150811,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
845,3180000,PHMA220213180034021200001,20210928,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
846,3120000,PHMA218853120033021200001,19740325,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
hospital = hospital[hospital['업태구분명'] == '종합병원']
hospital

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,진료과목내용명,지정취소일자,완화의료지정형태,완화의료담당부서명,구급차특수,구급차일반,총인원,구조사수,허가병상수,최초지정일자
9,3000000,PHMA219773000034021200001,19770401,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,3000000,PHMA219873000034021200001,19870424,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,3230000,PHMA219713230034021201559,19710220,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,3010000,PHMA219763010033021200001,19760310,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,3030000,PHMA219723030033021200001,19720424,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,3060000,PHMA220113060034021200001,20110309,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,3240000,PHMA220063240033021200002,20060227,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,3050000,PHMA219713050034021200002,19711005,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,3050000,PHMA220023050034021200002,20020801,NaN,1,영업/정상,13,영업중,NaN,NaN,...,내과 신경과 정신건강의학과 외과 정형외과 마취통증의학과 산부인과 소아청소년과 안과 ...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
110,3050000,PHMA219763050034021200001,19660107,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
hospital.columns

Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '의료기관종별명', '의료인수', '입원실수',
       '병상수', '총면적', '진료과목내용', '진료과목내용명', '지정취소일자', '완화의료지정형태', '완화의료담당부서명',
       '구급차특수', '구급차일반', '총인원', '구조사수', '허가병상수', '최초지정일자'],
      dtype='object')

In [7]:
hospital = hospital[['사업장명','도로명주소']]
hospital.isnull().sum()

사업장명     0
도로명주소    0
dtype: int64

In [8]:
hospital = hospital.reset_index(drop=True)

In [9]:
for i in range(len(hospital)):
    hospital['도로명주소'][i] = re.sub("\(.*\)|\s-\s.*", '', hospital['도로명주소'][i])
    print(hospital['도로명주소'][i])

서울특별시 종로구 새문안로 9 적십자병원 
서울특별시 종로구 통일로 256 
서울특별시 송파구 송이로 123 국립경찰병원 
서울특별시 중구 마른내로 9 
서울특별시 성동구 왕십리로 222-1 
서울특별시 중랑구 신내로 156 
서울특별시 강동구 동남로 892 
서울특별시 동대문구 경희대로 23 
서울특별시 동대문구 무학로 124 
서울특별시 동대문구 망우로 82 
서울특별시 광진구 자양로 85 
서울특별시 양천구 신정이펜1로 20 
서울특별시 강동구 진황도로61길 53 
서울특별시 구로구 경인로 427 구로성심병원 
서울특별시 영등포구 시흥대로 657 
서울특별시 강북구 도봉로 301 
서울특별시 동작구 보라매로5길 20 
서울특별시 노원구 한글비석로 68 을지병원 
서울특별시 은평구 통일로 1021 
서울특별시 은평구 통일로 873 
서울특별시 동대문구 왕산로 259 서울성심병원 
서울특별시 강서구 공항대로 389 부민병원 
서울특별시 중랑구 망우로 511 
서울특별시 강서구 하늘길 70 
서울특별시 강서구 공항대로 260 이화여자대학교 제2부속병원 
서울특별시 강남구 일원로 81 
서울특별시 강서구 강서로 295 
서울특별시 구로구 구로동로 148 고려대부속구로병원 
서울특별시 종로구 대학로 101 
서울특별시 영등포구 영등포로36길 13 
서울특별시 중구 을지로 245 
서울특별시 영등포구 여의대방로53길 22 
서울특별시 영등포구 도림로 156 명지성모병원 
서울특별시 서초구 반포대로 222 
서울특별시 용산구 대사관로 59 
서울특별시 강동구 성안로 150 
서울특별시 광진구 능동로 120-1 
서울특별시 양천구 안양천로 1071 
서울특별시 도봉구 우이천로 308 
서울특별시 영등포구 63로 10 여의도성모병원 
서울특별시 금천구 시흥대로 244 
서울특별시 종로구 새문안로 29 
서울특별시 영등포구 신길로 1 
서울특별시 관악구 남부순환로 1636 양지병원 
서울특별시 동작구 흑석로 102 
서울특별시 노원구 노원로 75 한국원자력의학원 
서울특별

In [11]:
hospital.head()

,사업장명,도로명주소
0,서울적십자병원,서울특별시 종로구 새문안로 9 적십자병원
1,세란병원,서울특별시 종로구 통일로 256
2,경찰병원,서울특별시 송파구 송이로 123 국립경찰병원
3,인제대학교 서울백병원,서울특별시 중구 마른내로 9
4,한양대학교병원,서울특별시 성동구 왕십리로 222-1


In [12]:
address = hospital['도로명주소']

In [13]:
geolocator = Nominatim(user_agent= 'South Korea')

In [14]:
lat = []
lng = []
for i in range(len(address)):
    try:
        location = geolocator.geocode(address[i])
        lat.append(location.latitude)
        lng.append(location.longitude)
    except:
        lng.append('d')
        lat.append('d')
        print(i, 'no')

0 no
2 no
20 no
21 no
24 no
27 no
41 no
45 no
46 no
48 no
50 no
53 no
54 no


In [15]:
hospital['lat'] = lat
hospital['lng'] = lng

- 서울 적십자 병원 : 37.56708343470264, 126.96700743481927
- 서울 경찰병원 : 37.496560640411545, 127.12347299572586
- 부민 병원 : 37.55703956921459, 126.85093772687601
- 이화여대 제2부속병원 : 37.56001966127979, 126.83650696038455
- 고려대 부속구로병원 : 37.492438608250254, 126.88497998454596
- 한국원자력의학원 : 37.62963710706514, 127.08263606920653
- 동신병원 : 37.58165489935458, 126.93670369804113
- 고려대병원 : 37.587152103447835, 127.02647582502688
- 홍익병원 : 37.52857101592637, 126.86360816920406

- geocoder로 찾을 수 없던 것들은 몇개 되지않아 수동으로 입력

In [17]:
hospital.loc[0,'lat'] = 37.56708343470264
hospital.loc[0,'lng'] = 126.96700743481927
hospital.loc[2,'lat'] = 37.496560640411545
hospital.loc[2,'lng'] = 127.12347299572586
hospital.loc[21,'lat'] = 37.55703956921459
hospital.loc[21,'lng'] = 126.85093772687601
hospital.loc[24,'lat'] = 37.56001966127979
hospital.loc[24,'lng'] = 126.83650696038455
hospital.loc[27,'lat'] = 37.492438608250254
hospital.loc[27,'lng'] = 126.88497998454596
hospital.loc[45,'lat'] = 37.62963710706514
hospital.loc[45,'lng'] = 127.08263606920653
hospital.loc[48,'lat'] = 37.58165489935458
hospital.loc[48,'lng'] = 126.93670369804113
hospital.loc[53,'lat'] = 37.587152103447835
hospital.loc[53,'lng'] = 127.02647582502688
hospital.loc[54,'lat'] = 37.52857101592637
hospital.loc[54,'lng'] = 126.86360816920406

In [18]:
hospital

,사업장명,도로명주소,lat,lng
0,서울적십자병원,서울특별시 종로구 새문안로 9 적십자병원,37.567083,126.967007
1,세란병원,서울특별시 종로구 통일로 256,37.57825,126.95416
2,경찰병원,서울특별시 송파구 송이로 123 국립경찰병원,37.496561,127.123473
3,인제대학교 서울백병원,서울특별시 중구 마른내로 9,37.563845,126.996236
4,한양대학교병원,서울특별시 성동구 왕십리로 222-1,37.555344,127.046663
5,서울특별시서울의료원,서울특별시 중랑구 신내로 156,37.613077,127.099211
6,강동경희대학교의대병원,서울특별시 강동구 동남로 892,37.555183,127.156584
7,경희대학교병원,서울특별시 동대문구 경희대로 23,37.593877,127.051872
8,서울특별시 동부병원,서울특별시 동대문구 무학로 124,37.578329,127.03036
9,삼육서울병원,서울특별시 동대문구 망우로 82,37.587489,127.064495


In [19]:
hospital['구'] = hospital['도로명주소'].str.split(" ").str[1]
hospital

,사업장명,도로명주소,lat,lng,구
0,서울적십자병원,서울특별시 종로구 새문안로 9 적십자병원,37.567083,126.967007,종로구
1,세란병원,서울특별시 종로구 통일로 256,37.57825,126.95416,종로구
2,경찰병원,서울특별시 송파구 송이로 123 국립경찰병원,37.496561,127.123473,송파구
3,인제대학교 서울백병원,서울특별시 중구 마른내로 9,37.563845,126.996236,중구
4,한양대학교병원,서울특별시 성동구 왕십리로 222-1,37.555344,127.046663,성동구
5,서울특별시서울의료원,서울특별시 중랑구 신내로 156,37.613077,127.099211,중랑구
6,강동경희대학교의대병원,서울특별시 강동구 동남로 892,37.555183,127.156584,강동구
7,경희대학교병원,서울특별시 동대문구 경희대로 23,37.593877,127.051872,동대문구
8,서울특별시 동부병원,서울특별시 동대문구 무학로 124,37.578329,127.03036,동대문구
9,삼육서울병원,서울특별시 동대문구 망우로 82,37.587489,127.064495,동대문구
